In [ ]:
from lobster.callbacks import CalmLinearProbeCallback

/homefs/home/sorenst1/code/prescient/lobster_internal/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:716: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/homefs/home/sorenst1/code/prescient/lobster_internal/.venv/lib/python3.12/site-packages/flash_attn/ops/triton/layer_norm.py:984: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/homefs/home/sorenst1/code/prescient/lobster_internal/.venv/lib/python3.12/site-packages/flash_attn/ops/triton/layer_norm.py:1043: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/homefs/home/sorenst1/code/prescient/lobster_internal/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:716: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
callback = CalmLinearProbeCallback(max_length=512)#, species=["hsapiens", "ecoli", "scerevisiae"])

In [ ]:
callback.tasks

{'function_bp',
 'function_cc',
 'function_mf',
 'localization',
 'meltome',
 'protein_abundance',
 'solubility',
 'transcript_abundance'}

In [ ]:
callback.species

In [ ]:
# model loading
from lobster.model._utils_checkpoint import get_ume_checkpoints

/homefs/home/sorenst1/code/prescient/lobster_internal/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:829: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/homefs/home/sorenst1/code/prescient/lobster_internal/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:829: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
get_ume_checkpoints()

2025-08-28 21:23:12,997 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials


{'ume-medium-base-480M': 's3://prescient-lobster/ume/runs/2025-06-12T16-47-59/epoch=0-step=39000-val_loss=0.5387.ckpt',
 'ume-small-base-90M': 's3://prescient-lobster/ume/runs/2025-06-29T00-32-13/epoch=2-step=109000-val_loss=0.0000.ckpt',
 'ume-mini-base-12M': 's3://prescient-lobster/ume/runs/2025-06-29T00-35-21/step-250000.ckpt',
 'ume-large-base-870M': 's3://prescient-lobster/ume/runs/2025-08-12T13-34-55/epoch=1-step=112500-val_loss=0.9124.ckpt',
 'ume-large-base-740M': 's3://prescient-lobster/ume/runs/2025-07-02T13-34-10/epoch=1-step=99500-val_loss=0.0024.ckpt'}

In [ ]:
# Sequence length statistics by CALM dataset (using CalmPropertyDataset)
from lobster.datasets import CalmPropertyDataset
import pandas as pd

# Tasks to summarize
base_tasks = [
    "meltome",
    "solubility",
    "localization",
]
species_tasks = {
    "protein_abundance": ["hsapiens", "ecoli", "scerevisiae"],
    "transcript_abundance": ["hsapiens", "ecoli", "scerevisiae"],
}
function_tasks = ["function_bp", "function_cc", "function_mf"]


def summarize_lengths(ds: CalmPropertyDataset) -> dict:
    input_col = ds.columns[0]
    lengths = ds.data[input_col].astype(str).str.len()
    return {
        "n": int(len(lengths)),
        "mean": float(lengths.mean()),
        "median": float(lengths.median()),
        "min": int(lengths.min()),
        "max": int(lengths.max()),
    }

rows = []
# Base tasks
for t in base_tasks:
    ds = CalmPropertyDataset(t)
    stats = summarize_lengths(ds)
    rows.append({"dataset": t, **stats})

# Species-specific tasks
for t, species_list in species_tasks.items():
    for sp in species_list:
        ds = CalmPropertyDataset(t, species=sp)
        stats = summarize_lengths(ds)
        rows.append({"dataset": f"{t}_{sp}", **stats})

# Function tasks
for t in function_tasks:
    ds = CalmPropertyDataset(t)
    stats = summarize_lengths(ds)
    rows.append({"dataset": t, **stats})

seq_len_df = pd.DataFrame(rows).sort_values("dataset").reset_index(drop=True)
seq_len_df


,dataset,n,mean,median,min,max
0,function_bp,452,7338.661504,1872.5,327,246282
1,function_cc,562,5335.745552,2145.0,184,206310
2,function_mf,369,7972.073171,2566.0,102,209955
3,localization,5696,1405.479986,1122.0,72,14910
4,meltome,14772,1519.353371,1146.0,6,80781
5,protein_abundance_ecoli,3627,951.875103,840.0,126,7077
6,protein_abundance_hsapiens,11789,1654.004920,1263.0,156,16791
7,protein_abundance_scerevisiae,4937,1493.976301,1230.0,105,14733
8,solubility,5222,1688.710456,1302.0,135,16791
9,transcript_abundance_ecoli,3528,970.506803,858.0,117,7077
